In [11]:
from ngsolve import *
from ngsolve.webgui import Draw
from netgen.occ import unit_square
from time import sleep, time

maxh = 0.15

mesh = unit_square.GenerateMesh(maxh=maxh)
for l in range(4): 
    mesh.Refine()
    maxh /= 2
mesh = Mesh(mesh)
# Draw (mesh)

In [2]:
order=2   # order=1 or order=2 is supported

tau = maxh / (5*order)
tend = 2
u0 = exp(-50**2*( (x-0.5)**2 + (y-0.5)**2))
v0 = 0

fes = H1LumpingFESpace(mesh, order=order)  
u,v = fes.TnT()

mform = u*v*dx(intrules=fes.GetIntegrationRules())
aform = grad(u)*grad(v)*dx

m = BilinearForm(mform, diagonal=True).Assemble()
a = BilinearForm(aform).Assemble()
minv = m.mat.Inverse(fes.FreeDofs())  

# or, in short:
# minv = fes.Mass(rho=1).Inverse()

In [4]:
gfu = GridFunction(fes)
gfu.Set(u0)

scene = Draw(gfu, order=order, deformation=True, scale=3, euler_angles=[-60,0,-40])
sleep (2)
unew = gfu.vec.CreateVector()
uold = gfu.vec.CreateVector()
uold.data = gfu.vec

with TaskManager(): 
    for n in range(int(tend/tau)):
        unew.data = 2*gfu.vec - uold 
        unew.data -= tau**2 * minv@a.mat * gfu.vec
        uold.data = gfu.vec
        gfu.vec.data = unew.data
        if n % 100 == 0:
            scene.Redraw()

scene.Redraw()

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {'camera': {'euler_angles': [-…

## Try gaussian initial conditions

In [5]:
order=2   # order=1 or order=2 is supported

tau = maxh / (5*order)
tend = 2
k = 3
u0 = sin(k*y)*exp(-25*(x*x + y*y))
v0 = k*cos(k*y)*exp(-25*(x*x + y*y))


fes = H1LumpingFESpace(mesh, order=order)  
u,v = fes.TnT()

mform = u*v*dx(intrules=fes.GetIntegrationRules())
aform = grad(u)*grad(v)*dx

m = BilinearForm(mform, diagonal=True).Assemble()
a = BilinearForm(aform).Assemble()
minv = m.mat.Inverse(fes.FreeDofs())  

# or, in short:
# minv = fes.Mass(rho=1).Inverse()

In [16]:
gfu = GridFunction(fes)
gfu.Set(u0)

scene = Draw(gfu, order=order, deformation=True, scale=3, euler_angles=[-60,0,-40])
sleep (2)
unew = gfu.vec.CreateVector()
uold = gfu.vec.CreateVector()
uold.data = gfu.vec

with TaskManager(): 
    for n in range(int(tend/tau)):
        unew.data = 2*gfu.vec - uold 
        unew.data -= tau**2 * minv@a.mat * gfu.vec
        uold.data = gfu.vec
        gfu.vec.data = unew.data
        if n % 100 == 0:
            scene.Redraw()

scene.Redraw()

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {'camera': {'euler_angles': [-…

In [13]:
order=2   # order=1 or order=2 is supported

tau = maxh / (5*order)
tend = 2
u0 = exp(-50**2*( (x-0.5)**2 + (y-0.5)**2))
v0 = 0

fes = H1LumpingFESpace(mesh, order=order)
u, v = fes.TnT()

mform = 1 * u*v*dx(intrules=fes.GetIntegrationRules())  # no IfPos here
m = BilinearForm(mform, diagonal=True).Assemble()

aform = grad(u)*grad(v)*dx
a = BilinearForm(aform).Assemble()

minv = m.mat.Inverse(fes.FreeDofs())  # invert diagonal M

u0 = exp(-50**2*((x-0.5)**2 + (y-0.5)**2))  # same as your code
gf_n1 = GridFunction(fes)
gf_n1.Set(u0)

unew = gf_n1.vec.CreateVector()
uold = gf_n1.vec.CreateVector()
uold.data = gf_n1.vec

nsteps = int(tend / tau)
for n in range(nsteps):
    unew.data = 2*gf_n1.vec - uold
    unew.data -= tau**2 * minv@a.mat * gfu.vec
    uold.data = gf_n1.vec
    gf_n1.vec.data = unew.data


In [14]:
fes = H1LumpingFESpace(mesh, order=order)
u, v = fes.TnT()

n2 = IfPos(x, 4, 1)
mform = n2 * u*v*dx(intrules=fes.GetIntegrationRules())
m = BilinearForm(mform, diagonal=True).Assemble()

aform = grad(u)*grad(v)*dx
a = BilinearForm(aform).Assemble()

minv = m.mat.Inverse(fes.FreeDofs())

# same initial condition
gfn2_0 = exp(-50**2*((x-0.5)**2 + (y-0.5)**2))
gf_n2 = GridFunction(fes)
gf_n2.Set(gfn2_0)

unew = gf_n2.vec.CreateVector()
uold = gf_n2.vec.CreateVector()
uold.data = gf_n2.vec

nsteps = int(tend / tau)
for n in range(nsteps):
    unew.data = 2*gf_n2.vec - uold
    unew.data -= tau**2 * minv@a.mat * gfu.vec
    uold.data = gf_n2.vec
    gf_n2.vec.data = unew.data

# Done. gf_n2 now holds the final solution with piecewise n^2.


In [15]:
diff = GridFunction(fes)
diff.vec.data = gf_n2.vec - gf_n1.vec
Draw(diff, mesh, "difference_n2")

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.25…

BaseWebGuiScene

## Try problems in 3D

In [19]:
maxh = 0.2
ng3dmesh = unit_cube.GenerateMesh(maxh=maxh)
mesh = Mesh(ng3dmesh)
order = 1
tau = maxh / (5*order)
tend = 1.5
u0 = exp(-50**2 * ((x-0.5)**2 + (y-0.5)**2 + (z-0.5)**2))

fes = H1LumpingFESpace(mesh, order=order)
u, v = fes.TnT()

mform = u*v*dx(intrules=fes.GetIntegrationRules())
m = BilinearForm(mform, diagonal=True).Assemble()

aform = grad(u)*grad(v)*dx
a = BilinearForm(aform).Assemble()

minv = m.mat.Inverse(fes.FreeDofs())  # diagonal inverse

gfu = GridFunction(fes)
gfu.Set(u0)

scene = Draw(gfu, mesh, order=order, 
             deformation=True, scale=1, 
             euler_angles=[-60,0,-40])
sleep(1)

unew = gfu.vec.CreateVector()
uold = gfu.vec.CreateVector()

uold.data = gfu.vec

nsteps = int(tend / tau)
with TaskManager():
    for n in range(nsteps):
        # u^{n+1} = 2*u^n - u^{n-1} - τ^2 * M^-1 * A * u^n
        tempvec = gfu.vec.CreateVector()
        tempvec.data = a.mat * gfu.vec
        unew.data = 2*gfu.vec - uold
        unew.data -= tau**2 * (minv * tempvec)
        # shift for next iteration
        uold.data = gfu.vec
        gfu.vec.data = unew.data
        if n % 50 == 0:
            scene.Redraw()

scene.Redraw()


WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {'camera': {'euler_angles': [-…